## Real estate investment in Milan

In [1]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
sale_raw<-read.csv2("idealista_buy.csv", as.is=T, sep=",")
rent_raw<-read.csv2("idealista_rent.csv", as.is=T, sep=",")

In [3]:
head(rent_raw)

,elevator,rooms,address,price,size
1,con ascensore,NA,"Monolocale in corso di porta nuova, 52, Moscova-Repubblica, Milano",680,27
2,,2,"Bilocale in via privata catone, 29, Dergano, Milano",700,80
3,con ascensore,3,"Trilocale in via giuseppe garibaldi, 94, Cinisello, Cinisello Balsamo",800,88
4,con ascensore,2,"Bilocale in via luigi guanella, 4, Precotto, Milano",800,40
5,senza ascensore,3,"Trilocale in via privata galeno, 18, Precotto, Milano",950,85
6,con ascensore,2,"Bilocale in via beato angelico, 23, CittĂ Studi, Milano",1.000,70


In [4]:
rent_raw$price<- as.numeric(gsub("\\.", "", rent_raw$price))
sale_raw$price<- as.numeric(gsub("\\.", "", sale_raw$price))

rent_raw$size<- as.numeric(rent_raw$size)
sale_raw$size<- as.numeric(sale_raw$size)

rent_raw$price_sqm<-rent_raw$price/rent_raw$size
sale_raw$price_sqm<-sale_raw$price/sale_raw$size

In [5]:
sale_raw$elevator[!(sale_raw$elevator  %in% c(" con ascensore"," senza ascensore"))]<-NA
rent_raw$elevator[!(rent_raw$elevator  %in% c(" con ascensore"," senza ascensore"))]<-NA

sale_raw$elevator[sale_raw$elevator == " con ascensore"]<-1
sale_raw$elevator[sale_raw$elevator == " senza ascensore"]<-0
rent_raw$elevator[rent_raw$elevator == " con ascensore"]<-1
rent_raw$elevator[rent_raw$elevator == " senza ascensore"]<-0

sale_raw$elevator<-as.numeric(sale_raw$elevator)
rent_raw$elevator<-as.numeric(rent_raw$elevator)

In [6]:
sale_raw$area<-sapply(sale_raw$address,function(x) tail(unlist(strsplit(x, ",")),n=2)[1])
sale_raw$city<-sapply(sale_raw$address,function(x) tail(unlist(strsplit(x, ",")), n=1))
rent_raw$area<-sapply(rent_raw$address,function(x) tail(unlist(strsplit(x, ",")),n=2)[1])
rent_raw$city<-sapply(rent_raw$address,function(x) tail(unlist(strsplit(x, ",")), n=1))

In [7]:
sale_clean<-sale_raw %>% 
            filter(area!=city) %>%
            group_by(address,size, elevator, rooms, area, city) %>% 
            summarize(min_price_sqm=min(price_sqm),min_price=min(price),max_price_sqm=max(price_sqm),max_price=max(price),nr_ad=n())

rent_clean<-rent_raw %>% 
            filter(area!=city) %>%
            group_by(address,size, elevator, rooms, area, city) %>% 
            summarize(min_price_sqm=min(price_sqm),min_price=min(price),max_price_sqm=max(price_sqm),max_price=max(price),nr_ad=n())

In [8]:
x <- sale_clean$min_price_sqm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
sale_clean<-sale_clean[sale_clean$min_price_sqm > (qnt[1] - H),]
sale_clean<-sale_clean[sale_clean$min_price_sqm < (qnt[2] + H),]

x <- rent_clean$min_price_sqm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
rent_clean<-rent_clean[rent_clean$min_price_sqm > (qnt[1] - H),]
rent_clean<-rent_clean[rent_clean$min_price_sqm < (qnt[2] + H),]

In [39]:
sale_stat<-sale_clean %>%
           group_by(city,area) %>%
           summarize(sale_count=n(), avg_sale_price=mean(min_price_sqm),
                     sale_difficulty=mean(nr_ad)) %>%
           filter(sale_count>=20)

rent_stat<-rent_clean %>%
           group_by(city,area) %>%
           summarize(rent_count=n(), avg_rent_price=mean(min_price_sqm),
                     rent_difficulty=mean(nr_ad)) %>%
           filter(rent_count>=20)

compare<-inner_join(sale_stat,rent_stat, by=c("city","area"))

compare<-compare %>%
         mutate(rent_to_own_ratio=avg_sale_price/(avg_rent_price*12),sale_per_rent_count=sale_count/rent_count,
                gross_yield=avg_rent_price*12/avg_sale_price) %>%
         arrange(desc(gross_yield))

group_invest<- function(value){
    if (value<15.0) return("Recommended to buy")
    if (value>21.0) return("Recommended to rent")
    return ("To be evaluated")
}

        
difficulty<- function(value){
    if (value<1.05) return("Easy")
    if (value>1.1) return("Difficult")
    return ("Medium")
}
        
sale_group<- function(value){
    if (value<2500) return("Cheap")
    if (value>4000) return("Expensive")
    return ("Normal")
}
        
rent_group<- function(value){
    if (value<13) return("Cheap")
    if (value>16) return("Expensive")
    return ("Normal")
}
  
        
compare<-compare %>% 
         rowwise() %>% 
         mutate(investment= group_invest(rent_to_own_ratio), rent_difficulty_group=difficulty(rent_difficulty),
                sale_difficulty_group=difficulty(sale_difficulty), sale_price_group=sale_group(avg_sale_price),
                rent_price_group=rent_group(avg_rent_price))

compare %>%
        select(area,gross_yield,rent_to_own_ratio,rent_difficulty_group,sale_difficulty_group,
               rent_price_group,sale_price_group, investment)

,area,gross_yield,rent_to_own_ratio,rent_difficulty_group,sale_difficulty_group,rent_price_group,sale_price_group,investment
1,Turro,0.08117513,12.31904,Easy,Difficult,Normal,Cheap,Recommended to buy
2,Quarto Oggiaro-Euromilano,0.07737052,12.92482,Easy,Easy,Cheap,Cheap,Recommended to buy
3,CĂ Granda,0.0732123,13.65891,Difficult,Medium,Normal,Cheap,Recommended to buy
4,Rogoredo-Santa Giulia,0.07206793,13.8758,Difficult,Easy,Cheap,Cheap,Recommended to buy
5,Gorla,0.07100762,14.08299,Easy,Easy,Normal,Cheap,Recommended to buy
6,Corvetto,0.06980334,14.32596,Easy,Medium,Normal,Cheap,Recommended to buy
7,Varesina-Testori,0.06920326,14.45019,Medium,Medium,Normal,Cheap,Recommended to buy
8,Dergano,0.06788529,14.73073,Easy,Easy,Normal,Cheap,Recommended to buy
9,Baggio,0.06759183,14.79469,Easy,Easy,Cheap,Cheap,Recommended to buy
10,Chiesa Rossa,0.06550028,15.26711,Medium,Easy,Normal,Normal,To be evaluated


In [40]:
write.csv(compare, file="compare.csv")